In [1]:
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from gym import Env
from gym import spaces
import random
from IPython.display import clear_output
import os
import matplotlib
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pettingzoo import ParallelEnv
import seaborn as sns

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#device setup
device = "cpu"

import environment
from per import Transition, PrioritizedReplayBuffer
from dqn import DQN

In [2]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 9.4* math.pow(10, 9)], [8 * math.pow(10, 9), 8.5* math.pow(10, 9)], [6 * math.pow(10, 9), 6.2* math.pow(10, 9)]]

hop_states = []

for i in range (0,3):
    radar_states = []
    interval = (freq_agile_radar_versions[i][1] - freq_agile_radar_versions[i][0])/10
    for j in range(0,24):
        radar_states.append(int(freq_agile_radar_versions[i][0]+ interval * j))
    hop_states.append(radar_states)

jamming_bandwidths = [10e6, 25e6, 40e6]

In [3]:
env = environment.ParallelRadarJammingEnv(hop_states, jamming_bandwidths)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class ParallelDQNTrainer:
    def __init__(self, env, num_agents=6, batch_size=32, max_steps=200, buffer_size=160000,
                  beta_start=0.4, beta_end=1.0, beta_increment=0.0005):
        self.env = env
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.beta = self.beta_start
        self.replay_buffer = PrioritizedReplayBuffer(capacity=10000)
        self.batch_size = 32
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.9995
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.num_agents = num_agents
        self.buffer = PrioritizedReplayBuffer(buffer_size)
        self.beta_increment = beta_increment
        self.gamma = 0.95

        agent_id = env.agents[0]
        
        self.observation_space = env.observation_space(agent_id).shape[0]
        self.action_space = env.action_space[agent_id].n

        self.model = DQN(self.observation_space, self.action_space).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

    def update_beta(self, current_episode):
        if self.beta < self.beta_end:
            self.beta = self.beta + self.beta_increment
        else:
            self.beta = self.beta_end
        return self.beta


    def select_action(self, agent, state):
        """ Select action for a given agent using epsilon-greedy policy """
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.env.action_space[agent].n)
        else:
            return torch.argmax(self.compute_q_values(state)).item()

    def train_agent(self, states, actions, rewards, next_states, dones, indices, weights):
        """ Train agent using DQN and prioritize replay buffer """
        q_values = self.compute_q_values(states)
        target_q_values = self.compute_target_q_values(next_states, rewards, dones)
        
        # Ensure shapes are consistent
        action_indices = torch.tensor(actions, dtype=torch.long).to(self.device)
        q_values = q_values.gather(1, action_indices.unsqueeze(1)).squeeze(1)

        # Compute loss
        weights = torch.tensor(weights, dtype=torch.float32).to(self.device)
        loss = nn.SmoothL1Loss(reduction='none')(q_values, target_q_values)
        weighted_loss = (loss * weights).mean()

        self.optimizer.zero_grad()
        weighted_loss.backward()
        self.optimizer.step()

        # Update priorities in the buffer
        priorities = torch.abs(q_values - target_q_values).detach().cpu().numpy() + 1e-5
        self.replay_buffer.update_priorities(indices, priorities)
    
    def compute_q_values(self, states):
            """ Compute Q-values from the DQN model for the current states. """
            states = torch.tensor(states, dtype=torch.float32).to(self.device)
            q_values = self.model(states)  # Model returns Q-values for each action
            return q_values

    def compute_target_q_values(self, next_states, rewards, dones):
        """ Compute target Q-values using the Bellman equation. """
        next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).to(self.device)

        next_q_values = self.model(next_states)  # Get Q-values for next states
        target_q_values = rewards + (1 - dones) * self.gamma * next_q_values.max(dim=1)[0]
        return target_q_values

    def compute_loss(self, q_values, target_q_values, actions, weights):
        """ Compute loss function (e.g., Huber Loss) between predicted and target Q-values. """
        action_indices = torch.tensor(actions, dtype=torch.long).to(self.device)
        q_values = q_values.gather(1, action_indices.unsqueeze(1))  # Select Q-values for taken actions
        loss = nn.SmoothL1Loss(reduction='none')(q_values.squeeze(1), target_q_values)
        weighted_loss = loss * torch.tensor(weights, dtype=torch.float32).to(self.device)
        return weighted_loss.mean()

    def train(self, episodes=10000):
        for episode in range(episodes):
            self.update_beta(episode)

            total_rewards = 0
            state = self.env.reset()
            done = {agent_id: False for agent_id in self.env.agents}

            while not all(done.values()):
                actions = {}
                for agent_id in self.env.agents:
                    if not done[agent_id]:
                        actions[agent_id] = self.select_action(agent_id, state[agent_id])

                next_state, reward, done, _ = self.env.step(actions)

                for agent_id in self.env.agents:
                    self.replay_buffer.add(
                        state[agent_id],
                        actions[agent_id],
                        reward[agent_id],
                        next_state[agent_id],
                        done[agent_id],
                    )

                state = next_state
                total_rewards += sum(reward.values())

            if len(self.replay_buffer.buffer) > self.batch_size:
                states, actions, rewards, next_states, dones, indices, weights = self.replay_buffer.sample(
                    self.batch_size, beta=self.beta
                )
                self.train_agent(states, actions, rewards, next_states, dones, indices, weights)

            print(f"Episode {episode}: Total Rewards: {total_rewards}")


In [ ]:
trainer = ParallelDQNTrainer(env)
trainer.train(episodes=10000)

c:\Users\Abhim\miniconda3\envs\env1\lib\site-packages\pettingzoo\utils\env.py:358: UserWarning: Your environment should override the observation_space function. Attempting to use the observation_spaces dict attribute.
  warnings.warn(


Episode 0: Total Rewards: -40530
Episode 1: Total Rewards: -45140
Episode 2: Total Rewards: -35650
Episode 3: Total Rewards: -49150
Episode 4: Total Rewards: -43860
Episode 5: Total Rewards: -42140
Episode 6: Total Rewards: -40340
Episode 7: Total Rewards: -37640
Episode 8: Total Rewards: -45220
Episode 9: Total Rewards: -40480
Episode 10: Total Rewards: -30240
Episode 11: Total Rewards: -36770
Episode 12: Total Rewards: -37720
Episode 13: Total Rewards: -39140
Episode 14: Total Rewards: -41520
Episode 15: Total Rewards: -53880
Episode 16: Total Rewards: -45110
Episode 17: Total Rewards: -32680
Episode 18: Total Rewards: -40840
Episode 19: Total Rewards: -53920
Episode 20: Total Rewards: -39830
Episode 21: Total Rewards: -37850
Episode 22: Total Rewards: -39180
Episode 23: Total Rewards: -42220
Episode 24: Total Rewards: -42390
Episode 25: Total Rewards: -39930
Episode 26: Total Rewards: -47150
Episode 27: Total Rewards: -33250
Episode 28: Total Rewards: -38010
Episode 29: Total Reward

In [ ]:
# class ParallelDQNTrainer:
#     def __init__(self, env, num_agents=6, batch_size=32, max_steps=200, buffer_size=16000,
#                   beta_start=0.4, beta_end=1.0, beta_increment=0.0005):
#         self.env = env
#         self.num_agents = num_agents
#         self.batch_size = batch_size
#         self.max_steps = max_steps
#         self.buffer = PrioritizedReplayBuffer(buffer_size)
#         self.beta_start = beta_start
#         self.beta = beta_start
#         self.beta_end = beta_end
#         self.beta_increment = beta_increment
#         agent_id = env.agents[0]

#         # Initialize agents
#         self.agents = [ParallelDQNAgent(i, env.observation_space(agent_id).shape[0], 
#                                         env.action_space[agent_id].n, self.buffer) for i in range(self.num_agents)]
        
#     def update_beta(self, current_episode):
#         if self.beta < self.beta_end:
#             self.beta = self.beta + self.beta_increment
#         else:
#             self.beta = self.beta_end
#         return self.beta

#     def train_step(self):
#         # Reset environment
#         observations = self.env.reset()

#         done = {agent: False for agent in self.env.agents}
#         total_rewards = {agent: 0 for agent in self.env.agents}

#         while not all(done.values()):
#             actions = {}
#             for agent in self.env.agents:
#                 actions[agent] = self.agents[self.env.agent_name_mapping[agent]].select_action(observations[agent])

#             next_observations, rewards, done, info = self.env.step(actions)

#             # Store transitions in the buffer
#             for agent in self.env.agents:
#                 self.buffer.add(observations[agent], actions[agent], rewards[agent], next_observations[agent], done[agent])

#             total_rewards = {agent: total_rewards[agent] + rewards[agent] for agent in self.env.agents}
#             observations = next_observations
        
#         all_rewards = sum(total_rewards.values())

#         return all_rewards

#     def train(self, episodes=5000):
#         for episode in range(episodes):
#             total_rewards = self.train_step()
#             # Train agents
#             for agent in self.agents:
#                 agent.train(self.batch_size, beta=self.update_beta(episode))

#             # Optionally, render or track performance
#             print(f"Episode {episode}: Total Rewards: {total_rewards}")